## Imports

In [3]:
import pandas as pd
import numpy as np

## Expoloring Data Import and Manipulation

In [4]:
# The traing data is so big, we cannot bring it in all ar once (12000 * 61190)
# I think when training We hace to go portion by portion, keep the stats, free the mem and read the next part
# there might be a better way also,but we can test with a small portion for now
train_df = pd.read_csv('training.csv', header=None, nrows=200)

#Testing set is 6774 * 61189

## Counting Priors

In [14]:
column_names =  ['doc_id'] + list(range(1,61189)) + ['label']
prior_counts = [0] * 21
for data_chunk in pd.read_csv('training.csv', header=None, chunksize=200, names=column_names, usecols=['label']):
    for _, row in data_chunk.iterrows():
        prior_counts [row['label']] += 1


In [15]:
print(prior_counts)

[0, 483, 624, 622, 643, 602, 630, 618, 614, 649, 628, 646, 639, 626, 621, 637, 651, 580, 593, 467, 427]


In [8]:
train_df.columns = ['doc_id'] + list(range(1,61189)) + ['label']
print(train_df['label'].value_counts())

16    19
6     16
3     14
12    14
7     14
5     12
4     11
8     10
17    10
10     9
1      9
13     9
11     8
2      8
14     7
15     7
20     7
18     6
9      5
19     5
Name: label, dtype: int64
